In [78]:
%autosave 0
%matplotlib inline

Autosave disabled


In [79]:
import codecs
import pymarc
import sqlite3
import yaml
from collections import defaultdict
import random

In [80]:
INPUT_MARCXML_FILE = "/src/input/metadata/marc21.xml"
MARCXML_PARSER_CONF = "/src/input/metadata/parsing_conf.yaml"
INPUT_SAMPLE_SIZE = 10

### Task 1 - Count number of records

In [81]:
#Done - in /src/input/metadata/marcxml_parsing

### Task 2 - Tally frequency of tags

In [82]:
#Done - in /src/input/metadata/marcxml_parsing

### Task 3 - Parse marcxml

In [93]:
marc_records = pymarc.parse_xml_to_array(INPUT_MARCXML_FILE)
try: marc_records = random.sample(marc_records, INPUT_SAMPLE_SIZE)
except: pass

### Task 4 - Map records to dictionary

In [94]:
def load_parser_conf(conf_file):
    with open(conf_file, "r") as stream:
        conf = yaml.load(stream)
    return conf

def fix_uid_mapping(marc_field):
    if 'u' in marc_field:
        if 'z' in marc_field: 
            uid = marc_field["z"].split(":")[0]
            if uid is not None:
                marc_field["u"] = uid + ": "+ marc_field["u"]
        else: marc_field.delete_subfield("u")
    return marc_field

def map_marcxml_record(marc_record, conf):
    marc_dict = defaultdict(set)
    for entry in conf:
        (k, v), = entry.items()
        field, sub_field = k.split("$")
        for marc_field in marc_record.get_fields(field):
            if field == "856": marc_field = fix_uid_mapping(marc_field)
            marc_subfield = marc_field[sub_field]
            if marc_subfield is not None: 
                marc_dict[v].add(marc_subfield)
    return marc_dict

def map_marcxml_records(marc_records, conf_file):
    conf = load_parser_conf(conf_file)
    marc_records_tf = map(lambda record: map_marcxml_record(record, conf), marc_records)
    return list(marc_records_tf)

map_marcxml_records(marc_records, MARCXML_PARSER_CONF)

[defaultdict(set,
             {'control_cardinality': {'multi'},
              'control_id': {'.b35279126'},
              'date_created_approx': {'2013.'},
              'language_code': {'eng'},
              'location_code': {'u-at-we'},
              'location_division': {'Western Australia'},
              'location_name': {'30 Aberdeen Street (Perth, W.A.)'},
              'note_general': {'Building 1 of the Institute is opposite at 12 Aberdeen Street (the old building)',
               'Title supplied by photographer.'},
              'note_public': {'Central Institute of Technology, Building 2, 30 Aberdeen Street, Perth, 23 May 2013. At the intersection of Aberdeen and William Streets',
               'Central Institute of Technology, Building 2, 30 Aberdeen Street, Perth, 23 May 2013. Back of the building on Lipfert Street',
               'Central Institute of Technology, Building 2, 30 Aberdeen Street, Perth, 23 May 2013. Main entrance on Aberdeen Street',
               'C